In [33]:
import pandas as pd
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import re

from tqdm import tqdm_notebook as tqdm

In [2]:
grammar_list = pd.read_csv('grammars_list.csv')

In [62]:
pattern = re.compile(r'（.*）')

pattern.findall('案の定（あんのじょう）')[0][1:-1]

In [43]:
exclude_paran = lambda x: re.sub(r'（.*）','',x)

In [107]:
grammar_list.head(20)

,link,description,level
0,https://jlptsensei.com/learn-japanese-grammar/...,敢えて（あえて）,N1
1,https://jlptsensei.com/learn-japanese-grammar/...,あげく,N2
2,https://jlptsensei.com/learn-japanese-grammar/...,上げる（あげる）,N3
3,https://jlptsensei.com/learn-japanese-grammar/...,間（あいだ）,N4
4,https://jlptsensei.com/learn-japanese-grammar/...,間に（あいだに）,N4
5,https://jlptsensei.com/learn-japanese-grammar/...,あくまでも,N1
6,https://jlptsensei.com/learn-japanese-grammar/...,あまり,N3
7,https://jlptsensei.com/learn-japanese-grammar/...,あまりにも,N3
8,https://jlptsensei.com/learn-japanese-grammar/...,あまり～ない,N4
9,https://jlptsensei.com/learn-japanese-grammar/...,案の定（あんのじょう）,N1


In [102]:
idx = 806
link = grammar_list.iloc[idx]['link']
topic = grammar_list.iloc[idx]['description']

In [103]:
pattern = re.compile(r'（.*）')
hira_list = pattern.findall(topic)
hira_topic = None
if len(hira_list) > 0:
    hira_topic = hira_list[0][1:-1]

In [104]:
hira_topic

'ずにすむ'

In [5]:
link

'https://jlptsensei.com/learn-japanese-grammar/%e9%96%93%e3%81%ab-aida-ni-meaning/'

In [21]:
http = httplib2.Http()

In [87]:
_, response = http.request(link)

soup = BeautifulSoup(response)

In [54]:
usage_table = soup.find('table',{'class':'usage'})

In [58]:
usage_table.find_all('td')[0]

'3'

In [95]:
def get_usage_num_column(soup, topic):
    pattern = re.compile(r'（.*）')
    hira_list = pattern.findall(topic)
    hira_topic = None
    if len(hira_list) > 0:
        hira_topic = hira_list[0][1:-1]
    
    usage_table = soup.find('table',{'class':'usage'})
    max_column = 0
    position_id = -1
    for tr in usage_table.find_all('tr'):
        max_column = max(len(tr.find_all('td')), max_column)
        for idx, td in enumerate(tr.find_all('td')):
#             print(td.text)
            if td.get('rowspan') is not None:
                position_id = idx
                break
            if td.text is not None:
                if exclude_paran(topic) in exclude_paran(td.text):
                    position_id = idx
                    break
                if hira_topic is not None:        
                    if hira_topic in exclude_paran(td.text):
                        position_id = idx
                        break
        
    
    return {
        'num_col': max_column,
        'pos_idx': position_id
    }
    

In [96]:
get_usage_num_column(soup, topic)

{'num_col': 2, 'pos_idx': 1}

In [112]:
grammar_list[pd.DataFrame(datas)['pos_idx']==-1].head(20)

,link,description,level
15,https://jlptsensei.com/learn-japanese-grammar/...,ば,N4
16,https://jlptsensei.com/learn-japanese-grammar/...,ばいい,N3
18,https://jlptsensei.com/learn-japanese-grammar/...,ばそれまでだ / たらそれまでだ,N1
19,https://jlptsensei.com/learn-japanese-grammar/...,ばよかった,N3
20,https://jlptsensei.com/learn-japanese-grammar/...,ば～ほど,N3
30,https://jlptsensei.com/learn-japanese-grammar/...,べからず / べからざる,N1
38,https://jlptsensei.com/learn-japanese-grammar/...,ぶりに,N3
39,https://jlptsensei.com/learn-japanese-grammar/...,ちゃいけない・じゃいけない,N5
41,https://jlptsensei.com/learn-japanese-grammar/...,ちっとも～ない,N2
43,https://jlptsensei.com/learn-japanese-grammar/...,だ・です,N5


In [99]:
# datas = []
for i in tqdm(range(len(grammar_list))):
    if datas[i]['pos_idx'] != -1: continue
    link = grammar_list.iloc[i]['link']
    topic = grammar_list.iloc[i]['description']
    
    _, response = http.request(link)
    soup = BeautifulSoup(response)
    
    dat = get_usage_num_column(soup, topic)
    datas[i] = dat

<ipython-input-99-d1a747fc6418>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(grammar_list))):


In [28]:
get_usage_num_column(soup, topic)

{'num_col': 3, 'pos_idx': 1}

In [10]:
usage_table.find_all('td')

[<td>Verb (casual, non-past)</td>,
 <td rowspan="4">間に</td>,
 <td>Noun + の</td>,
 <td>い-adjective + い</td>,
 <td>な-adjective + な</td>]

In [20]:
a = 'asd '

In [21]:
a.strip()

'asd'

In [18]:
def convert_to_formula(st: str):
    convert_dict = {
        'Noun': '<N>',
        'い-adjective': '<i-adj>',
        'な-adjective': '<na-adj>'
    }
    tokens = [x.strip() for x in trimst.split('+')]
    for 
    print(tokens)

In [19]:
convert_to_formula('<td>Noun + の</td>')

['<td>Noun ', ' の</td>']
